In [1]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

matrix = np.array([
    [0.087, 0.084, 0.115, 0.111],
    [0.066, 0.079, 0.111, 0.102],
    [0.044, 0.086, 0.085, 0.088],
    [0.088, 0.086, 0.244, 0.078],
    [0.285, 0.063, 0.122, 0.068],
    [0.040, 0.134, 0.018, 0.144],
    [0.030, 0.096, 0.020, 0.148],
    [0.039, 0.130, 0.112, 0.105],
    [0.109, 0.103, 0.124, 0.089], 
    [0.014, 0.139, 0.040, 0.069]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10']
criteria = ['C1', 'C2', 'C3', 'C4']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.25377794, 0.35076977, 0.39287723, 0.00257506])

# NORMALIZATION FUNCTION (min-max normalization)
def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

# CODAS METHOD
def codas(matrix, weights, criteria_types, sites, criteria, tau=0.02):
    norm_matrix = normalize_matrix(matrix, criteria_types)
    weighted_matrix = norm_matrix * weights

    # Negative-ideal solution (worst for each criterion)
    neg_ideal = np.min(weighted_matrix, axis=0)

    # Euclidean and Taxicab distances from negative-ideal
    euclidean = np.sqrt(((weighted_matrix - neg_ideal) ** 2).sum(axis=1))
    taxicab = np.abs(weighted_matrix - neg_ideal).sum(axis=1)

    # Assessment score
    # If difference in Euclidean distance between two alternatives < tau, use Taxicab distance as tie-breaker
    scores = np.zeros(len(sites))
    for i in range(len(sites)):
        count = 0
        for j in range(len(sites)):
            if i == j:
                continue
            diff = euclidean[i] - euclidean[j]
            if diff > tau:
                count += 1
            elif abs(diff) <= tau:
                if taxicab[i] > taxicab[j]:
                    count += 1
        scores[i] = count

    # Alternatively, for ranking, you can use the Euclidean distance directly (higher is better)
    df = pd.DataFrame({
        'Euclidean': euclidean,
        'Taxicab': taxicab,
        'CODAS Score': euclidean,  # For simple ranking, use Euclidean distance
        'Rank': rankdata(-euclidean, method='min')
    }, index=sites).sort_values('Rank')

    return df

result_df = codas(matrix, weights, criteria_types, sites, criteria)
print(result_df)


        Euclidean   Taxicab  CODAS Score  Rank
Site4    0.412824  0.568650     0.412824     1
Site10   0.352849  0.389047     0.352849     2
Site8    0.350537  0.497243     0.350537     3
Site6    0.328605  0.354487     0.328605     4
Site5    0.311592  0.434571     0.311592     5
Site9    0.275596  0.458524     0.275596     6
Site1    0.206163  0.335292     0.206163     7
Site2    0.184291  0.285307     0.184291     8
Site3    0.160075  0.251364     0.160075     9
Site7    0.153104  0.173343     0.153104    10
